In [1]:
# Importaciones
import pandas as pd
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
import numpy as np
from sklearn import preprocessing
import seaborn as sns
from datetime import datetime
from IPython.core.display import display, HTML
from scipy import stats
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# data=pd.read_csv(r'C:\Users\alexb\Documents\Facultad\UBA\Modelo_PGSM_3T\Datos\20210811 - Segunda corrida de datos\To_FS_20210913_2137_3T.csv', delimiter=',', sep='\n', decimal='.', index_col='Time')
data = pd.read_pickle('./To_FS_20211121.pkl')
data.head()

,T_Lam_9,MG_NIR_Salida_Ext,Conc_P3-11,TT-35-GO,T_Lam_9_55,Humedad_Semilla,T_Mat_entrada,T_Solvente,Q_Solvente,Ritmo_Molienda,Q_Miscela,Vel_Extractor,Conc_Miscela,Solvente_NIR_Salida_Ext,LT-17,TT-35-GI
Time,,,,,,,,,,,,,,,,
2021-10-07 15:00:00,0.086037,0.725236,0.02962,0.108910,0.0,0.534636,0.324695,0.443189,0.325671,0.329397,0.282113,0.328315,0.312149,0.605080,0.505380,0.269928
2021-10-07 15:03:00,0.089563,0.726583,0.02962,0.102093,0.0,0.534636,0.340239,0.313222,0.307250,0.329384,0.281493,0.327030,0.318091,0.591038,0.496463,0.281288
2021-10-07 15:06:00,0.093035,0.826301,0.02962,0.078385,0.0,0.533409,0.385105,0.314746,0.333278,0.329414,0.280784,0.327030,0.325121,0.584253,0.498974,0.269985
2021-10-07 15:09:00,0.096127,0.783242,0.02962,0.078234,0.0,0.533918,0.442633,0.293762,0.317303,0.329656,0.280892,0.327030,0.328268,0.586444,0.510685,0.277349
2021-10-07 15:12:00,0.098893,0.785093,0.02962,0.077646,0.0,0.534636,0.460639,0.531045,0.309744,0.330145,0.281104,0.327030,0.328484,0.611053,0.531662,0.279702


In [5]:
X = data.drop(columns = 'MG_NIR_Salida_Ext')
y = data.MG_NIR_Salida_Ext

# SELECT FROM MODEL (SFM)

**SFM** o **SELECT FROM MODEL** es un método de selección de features que utiliza las estimaciones de un modelo (al igual que RFE). La diferencia es que en este caso la selección de las features se hace estableciendo un umbral para el atributo que se está evaluando (coeficientes o feature importances).

*Primer conjunto de variables seleccionado: Index(['Conc_P3-4_MVA', 'T_Lam_9_MVA', 'Conc_P1', 'T_Mat_entrada_MVA'], dtype='object')*

In [6]:
from sklearn.ensemble import GradientBoostingRegressor

In [7]:
from sklearn.feature_selection import SelectFromModel

In [8]:
sfm_selector = SelectFromModel(estimator = GradientBoostingRegressor())

In [9]:
sfm_selector.fit(X, y)

SelectFromModel(estimator=GradientBoostingRegressor())

In [10]:
X.columns[sfm_selector.get_support()]

Index(['T_Lam_9', 'Conc_P3-11', 'TT-35-GO', 'Q_Miscela', 'LT-17'], dtype='object')

# RECURSIVE FEATURE ELIMINATION (RFE)

**RFE** o **RECURSIVE FEATURE ELIMINATION** es un método de selección de características utilizado en ML en donde se elimina la característica menos importantes luego de entrenar de forma recursiva.

Se comienza entrenando en el conjunto completo de features y se obtiene la importancia de cada feature mediante los coeficientes del modelo. En base a eso, la feature con menos importancia es eliminada.
El proceso se repite hasta alcanzar el numero deseado de características.

*Primer conjunto de variables seleccionado: Index(['Conc_P3-4_MVA', 'T_Lam_9_MVA', 'Conc_P1', 'T_Mat_entrada_MVA'], dtype='object')*

In [11]:
from sklearn.feature_selection import RFE

In [15]:
rfe_selector = RFE(estimator = GradientBoostingRegressor(), n_features_to_select = 6, step = 1) # step es la cantidad de features que se descartan por iteración

In [16]:
rfe_selector.fit(X, y)

RFE(estimator=GradientBoostingRegressor(), n_features_to_select=6)

In [17]:
X.columns[rfe_selector.get_support()]

Index(['T_Lam_9', 'Conc_P3-11', 'TT-35-GO', 'T_Mat_entrada', 'Q_Miscela',
       'LT-17'],
      dtype='object')

# SEQUENTIAL FEATURE SELECTION (SFS)

**SFE** o **SEQUENTIAL FEATURE SELECTION** es un algorítmo de selección de features que utiliza el puntaje obtenido con el CV set como estimador para seleccionar las mejores features.
Existen dos variantes de este algorítmo: SFS-Forward y SFS-Backward.

En el caso de SFS-Forward, se comienza con una única feature (la que mejor resultado brinda por si sola en el CV set). Luego se van seleccionando de a una, las variables que mejor se desempeñan, hasta alcanzar el numero de características deseadas.

Notar que en este caso **no se usan los atributos del modelo: coef_ ni feature_importances_**. Por otro lado, este algorítmo es más lento, ya que resulta de entrenar el modelo múltiples veces.

*Primer conjunto de variables seleccionado: Index(['Vel_Extractor', 'T_Mat_entrada_MVA', 'TT-35-GI', 'T_Solvente_MVA'], dtype='object')*

In [18]:
from sklearn.feature_selection import SequentialFeatureSelector

In [19]:
sfs_selector = SequentialFeatureSelector(estimator = GradientBoostingRegressor(), n_features_to_select = 6, cv = 4, direction = 'forward')

In [20]:
sfs_selector.fit(X, y)

SequentialFeatureSelector(cv=4, estimator=GradientBoostingRegressor(),
                          n_features_to_select=6)

In [21]:
X.columns[sfs_selector.get_support()]

Index(['Conc_P3-11', 'T_Lam_9_55', 'Humedad_Semilla', 'T_Solvente',
       'Ritmo_Molienda', 'TT-35-GI'],
      dtype='object')

In [42]:
sfs_selector.get_params()

{'cv': 4,
 'direction': 'forward',
 'estimator__alpha': 0.9,
 'estimator__ccp_alpha': 0.0,
 'estimator__criterion': 'friedman_mse',
 'estimator__init': None,
 'estimator__learning_rate': 0.1,
 'estimator__loss': 'ls',
 'estimator__max_depth': 3,
 'estimator__max_features': None,
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_iter_no_change': None,
 'estimator__random_state': None,
 'estimator__subsample': 1.0,
 'estimator__tol': 0.0001,
 'estimator__validation_fraction': 0.1,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': GradientBoostingRegressor(),
 'n_features_to_select': 4,
 'n_jobs': None,
 'scoring': None}